In [42]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import requests

In [43]:
class AxieCriterion:
    def __init__(self, query):
        url, params = query.split('?')
        classes = []
        parts = []
        speeds = []
        hps = []
        skills = []
        morales = []
        for param in params.split('&'):
            attr, val = param.split('=')
            if attr == 'class':
                classes.append(val)
            elif attr == 'part':
                parts.append(val)
            elif attr == 'speed':
                speeds.append(int(val))
            elif attr == 'hp':
                hps.append(int(val))
            elif attr == 'skill':
                skills.append(int(val))
            elif attr == 'morale':
                morales.append(int(val))
                
        speeds = sorted(speeds)
        hps = sorted(hps)
        skills = sorted(skills)
        morales = sorted(morales)
        
        self.body = {
            "operationName": "GetAxieBriefList",
            "query": "query GetAxieBriefList($auctionType: AuctionType, $criteria: AxieSearchCriteria, $from: Int, $sort: SortBy, $size: Int, $owner: String, $filterStuckAuctions: Boolean) {\n  axies(\n    auctionType: $auctionType\n    criteria: $criteria\n    from: $from\n    sort: $sort\n    size: $size\n    owner: $owner\n    filterStuckAuctions: $filterStuckAuctions\n  ) {\n    total\n    results {\n      ...AxieBrief\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment AxieBrief on Axie {\n  id\n  name\n  stage\n  class\n  breedCount\n  image\n  title\n  battleInfo {\n    banned\n    __typename\n  }\n  auction {\n    currentPrice\n    currentPriceUSD\n    __typename\n  }\n  parts {\n    id\n    name\n    class\n    type\n    specialGenes\n    __typename\n  }\n  __typename\n}\n",
            "variables": {
                "auctionType": "Sale",
                "criteria": {
                    "bodyShapes": None,
                    "breedable": None,
                    "breedCount": None,
                    "classes": classes,
                    "hp": [],
                    "morale": [],
                    "numJapan": None,
                    "numMystic": None,
                    "numXmas": None,
                    "parts": parts,
                    "pureness": None,
                    "purity": [],
                    "region": None,
                    "skill": [],
                    "speed": [],
                    "stages": None,
                    "title": None
                },
                "filterStuckAuctions": True,
                "from": 0,
                "owner": None,
                "size": 24,
                "sort": "PriceAsc"
            }
        }
        if len(speeds) == 2:
            self.body["variables"]["criteria"]["speed"] = speeds
        if len(hps) == 2:
            self.body["variables"]["criteria"]["hp"] = hps
        if len(skills) == 2:
            self.body["variables"]["criteria"]["skill"] = skills
        if len(morales) == 2:
            self.body["variables"]["criteria"]["morale"] = morales
        
    def marketplace(self):
        r = requests.post("https://graphql-gateway.axieinfinity.com/graphql", json=self.body)
        data = json.loads(r.text)['data']['axies']
        return data
    
query = 'https://marketplace.axieinfinity.com/axie/?class=Bird&speed=61&speed=61&auctionTypes=Sale'
ac = AxieCriterion(query)
ac.marketplace()

{'total': 23561,
 'results': [{'id': '4814027',
   'name': 'Sponge Bird',
   'stage': 4,
   'class': 'Bird',
   'breedCount': 0,
   'image': 'https://assets.axieinfinity.com/axies/4814027/axie/axie-full-transparent.png',
   'title': '',
   'battleInfo': {'banned': False, '__typename': 'AxieBattleInfo'},
   'auction': {'currentPrice': '17850000000000000',
    'currentPriceUSD': '44.83',
    '__typename': 'Auction'},
   'parts': [{'id': 'eyes-mavis',
     'name': 'Mavis',
     'class': 'Bird',
     'type': 'Eyes',
     'specialGenes': None,
     '__typename': 'AxiePart'},
    {'id': 'ears-pink-cheek',
     'name': 'Pink Cheek',
     'class': 'Bird',
     'type': 'Ears',
     'specialGenes': None,
     '__typename': 'AxiePart'},
    {'id': 'back-sponge',
     'name': 'Sponge',
     'class': 'Aquatic',
     'type': 'Back',
     'specialGenes': None,
     '__typename': 'AxiePart'},
    {'id': 'mouth-little-owl',
     'name': 'Little Owl',
     'class': 'Bird',
     'type': 'Mouth',
     'sp

In [44]:
def get_optimal_deck(addr):
    url = f"https://tracking.skymavis.com/battle-history?type=pvp&player_id={addr}"
    response = requests.request("GET", url)
    battles = json.loads(response.text)["battles"]

    deck_dict = {}
    most_freq = 0
    for battle in battles:
        team1 = tuple(sorted(battle['first_team_fighters']))
        if team1 in deck_dict:
            deck_dict[team1] += 1
        else:
            deck_dict[team1] = 1

        if most_freq < deck_dict[team1]:
            most_freq = deck_dict[team1]
            opt_deck = team1

        team2 = tuple(sorted(battle['second_team_fighters']))
        if team2 in deck_dict:
            deck_dict[team2] += 1
        else:
            deck_dict[team2] = 1

        if most_freq < deck_dict[team2]:
            most_freq = deck_dict[team2]
            opt_deck = team2
    return opt_deck, most_freq

In [48]:
def get_axie_for_sale(id):
    url = f'https://api.axie.technology/getaxies/{id}'
    response = requests.request("GET", url)
    axie = json.loads(response.text)
    url_parts =''.join([f'&part={part["id"]}' for part in axie['parts'] if not (part["id"].split('-')[0] in ['ears', 'eyes'])])
    del axie["stats"]['__typename']
    sorted_dict = sorted(axie["stats"].items(), key = lambda item: item[1])
    url_stats = ''
    for stat in sorted_dict[-1:]:
        url_stats += f'&{stat[0]}={stat[1]}&{stat[0]}=61'
    return f'https://marketplace.axieinfinity.com/axie/?class={axie["sireClass"]}{url_parts}{url_stats}&auctionTypes=Sale'

get_axie_for_sale(10825538)

'https://marketplace.axieinfinity.com/axie/?class=Mech&part=back-hero&part=mouth-nut-cracker&part=horn-arco&part=tail-nut-cracker&speed=49&speed=61&auctionTypes=Sale'

In [49]:
payload = {
    "operationName": "NewEthExchangeRate",
    "query": "query NewEthExchangeRate {\n  exchangeRate {\n    eth {\n      usd\n      __typename\n    }\n    __typename\n  }\n}\n",
    "variables": {}
}
r = requests.post("https://graphql-gateway.axieinfinity.com/graphql", json=payload)
ethusd = json.loads(r.text)['data']['exchangeRate']['eth']['usd']
ethusd

2504.29

### Track similar deck w/ ids

In [53]:
ids = [url.split('/')[-2] for url in
'''
https://marketplace.axieinfinity.com/axie/10878361/
https://marketplace.axieinfinity.com/axie/10878072/
'''.split('\n')[1:-1]]

def track_by_ids(ids):
    total_price = 0
    total_query = ''
    for i, id in enumerate(ids):
        query = get_axie_for_sale(id)
        ac = AxieCriterion(query)
        data = ac.marketplace()
        if len(data['results']) != 0:
            results = data['results'][0]

            price = round(float(results["auction"]["currentPriceUSD"]) / ethusd, 3)
            if i != 0:
                print(' +', price, "ETH", end='')
            else:
                print(price, "ETH", end='')
        else:
            query = 'n/a'
            price = 0

        total_query += query + '\n'
        total_price += price
    print(f' = {total_price:.3f} ETH')
    print(total_query)
track_by_ids(ids)

0.042 ETH + 0.042 ETH = 0.084 ETH
https://marketplace.axieinfinity.com/axie/?class=Bug&part=back-sandal&part=mouth-pincer&part=horn-parasite&part=tail-fish-snack&morale=57&morale=61&auctionTypes=Sale
https://marketplace.axieinfinity.com/axie/?class=Bug&part=back-scarab&part=mouth-pincer&part=horn-parasite&part=tail-fish-snack&morale=57&morale=61&auctionTypes=Sale



### Track deck w/ queries

In [51]:
queries = '''
 https://marketplace.axieinfinity.com/axie/?class=Dusk&part=back-green-thorns&part=mouth-tiny-turtle&part=horn-kestrel&part=tail-hot-butt&hp=53&hp=61&speed=50&speed=61&auctionTypes=Sale
https://marketplace.axieinfinity.com/axie/?class=Plant&part=back-watering-can&part=mouth-serious&part=horn-cactus&part=tail-hot-butt&hp=61&hp=61&auctionTypes=Sale
https://marketplace.axieinfinity.com/axie/?class=Bug&part=back-scarab&part=mouth-pincer&part=horn-parasite&part=tail-fish-snack&morale=55&morale=61&auctionTypes=Sale
'''.split()

def track_by_queries(queries):
    total_price = 0
    total_query = ''
    for i, query in enumerate(queries):
        ac = AxieCriterion(query)
        data = ac.marketplace()
        if len(data['results']) != 0:
            results = data['results'][0]

            price = round(float(results["auction"]["currentPriceUSD"]) / ethusd, 3)
            if i != 0:
                print(' +', price, "ETH", end='')
            else:
                print(price, "ETH", end='')
        else:
            query = 'n/a'
            price = 0

        total_query += query + '\n'
        total_price += price
    print(f' = {total_price:.3f} ETH')
    print(total_query)
track_by_queries(queries)

0.046 ETH + 0.034 ETH + 0.041 ETH = 0.121 ETH
https://marketplace.axieinfinity.com/axie/?class=Dusk&part=back-green-thorns&part=mouth-tiny-turtle&part=horn-kestrel&part=tail-hot-butt&hp=53&hp=61&speed=50&speed=61&auctionTypes=Sale
https://marketplace.axieinfinity.com/axie/?class=Plant&part=back-watering-can&part=mouth-serious&part=horn-cactus&part=tail-hot-butt&hp=61&hp=61&auctionTypes=Sale
https://marketplace.axieinfinity.com/axie/?class=Bug&part=back-scarab&part=mouth-pincer&part=horn-parasite&part=tail-fish-snack&morale=55&morale=61&auctionTypes=Sale



In [22]:
response = requests.request("GET", 'https://axie.zone/leaderboard')
html = response.text
trs = bs(html).find_all('tr')
leaderboard = []

for tr in trs[1:20]:
    tds = tr.find_all('td')
    rank = int(tds[0].text[1:])
    ron_addr = tds[1].find("a")["href"].split('=')[-1]
    name = tds[1].text
    axies = get_optimal_deck(ron_addr)[0]
    leaderboard.append([rank, ron_addr, name, axies[0], axies[1], axies[2]])
    print(rank, name)
    track_by_ids(axies)
    
df = pd.DataFrame(leaderboard, columns=['rank', 'ron_addr', 'name', 'axie#1', 'axie#2', 'axie#3'])
df

1 Mapo Tofu | fb.gg/tofuknightsPH
0.169 ETH + 0.125 ETH + 0.145 ETH = 0.439 ETH
https://marketplace.axieinfinity.com/axie/?class=Bird&part=back-balloon&part=mouth-peace-maker&part=horn-arco&part=tail-hare&speed=57&speed=61&auctionTypes=Sale
https://marketplace.axieinfinity.com/axie/?class=Plant&part=back-bidens&part=mouth-goda&part=horn-beech&part=tail-cattail&hp=58&hp=61&auctionTypes=Sale
https://marketplace.axieinfinity.com/axie/?class=Mech&part=back-furball&part=mouth-lam&part=horn-arco&part=tail-twin-tail&speed=50&speed=61&auctionTypes=Sale

2 TUP Baam
0.03 ETH + 0.051 ETH + 0.049 ETH = 0.130 ETH
https://marketplace.axieinfinity.com/axie/?class=Plant&part=back-shiitake&part=mouth-serious&part=horn-cactus&part=tail-carrot&hp=61&hp=61&auctionTypes=Sale
https://marketplace.axieinfinity.com/axie/?class=Plant&part=back-bidens&part=mouth-herbivore&part=horn-cactus&part=tail-cattail&hp=61&hp=61&auctionTypes=Sale
https://marketplace.axieinfinity.com/axie/?class=Aquatic&part=back-risky-beas

,rank,ron_addr,name,axie#1,axie#2,axie#3
0,1,0x53e06a192466cd29b1809a4c1eca3f746bd08d65,Mapo Tofu | fb.gg/tofuknightsPH,9689597,10352131,10521508
1,2,0xeb68215f550be82c0d7be10c38ff69c2d30cdb56,TUP Baam,8791724,9653888,10377089
2,3,0x4d9f7c8e438943ba2cccd3a24d91f7cf9b5a7cc6,Spriggan,369570,673113,6143639
3,4,0xa261cdf0942e79b1ca92b02e6ba166ae4798538b,fb.gg/KouseiPotatoSquad :P,10153722,10175055,10612886
4,5,0x47c73a4e7a93a4797b804f048fb983d6d492e1b8,Dave | The LNCN Mafia,4696326,8737652,10129601
5,6,0x2243205cbe1af6e8f40a64a3f6c028e92214c9bc,BIG CHIEF | USA + PH | Chan,2657205,6881219,9704497
6,7,0x07f539753f239dded678ba137a7cc20118130d84,PemPem | Ancient8,9818997,10564486,10580236
7,8,0x6e9664764dff76d25ffa312c6a0eb534e5c80215,Roblolux,6536128,9351632,10463066
8,9,0x3a492379eb13dff01d90ec1a7ae696f8955aa714,xSkyyy | Axie.gg,6209490,9878890,10499324
9,10,0x936064823bd7fe3c86b63629083b05ae9e13e2dd,Russ | POG,9356633,10445186,10596343
